In [75]:
import re
import json
from parse_bitstream_v2 import BitstreamParser
import hashlib
from collections import defaultdict


In [76]:
json_file_path = r"E:\workspace\parse_bitstream\parse_bitstream\压缩位流相关知识\MCF100T_v2.json"
# 读取JSON文件
with open(json_file_path, 'r') as file:
    json_file = json.load(file)

In [77]:
file_path = r"E:\workspace\parse_bitstream\parse_bitstream\data\system_wrapper_1_原版.rbt"
bit_parser = BitstreamParser(file_path, "False")

In [78]:
data_frame = bit_parser.rbt_data_content

In [79]:
# 获取特征值
# 这里传入content为list， 每个元素的数据类型为 content_type
def get_feature(content:list, content_type = "int"):
    if content_type == "str":
        # 此时是二进制字符串作为一个元素
        combined_string = ''.join(content)
        # 使用 hashlib 计算特征值（SHA256）
        feature_hash = hashlib.sha256(combined_string.encode()).hexdigest()
        return feature_hash
    elif content_type == "int":
        pass

In [101]:
# 数据帧特征值作为key
# value : 
# {   
#     "FAR": {
#         "frame_type": 0,
#         "region_type": 0,
#         "row_num": 0,
#         "col_num": 0
#     },
#     "index":{
#         "start_index":0,
#         "end_index":101
#     },
#     "feature": "123"
# }
# 每个row相互独立
data_frame_features_index = defaultdict(list)
start_index = 0
end_index = 101 # 这里是开区间，不包括101
cur_row = 0
frame_count = 0
all_frame_count = 0
for frame_type_key, frame_type_value in json_file.items():
    
    # type 0 type 1
    data_frame_features_index[frame_type_key] = {}
    
    # 遍历 frame_type 层
    for region_type_key, region_type_value in frame_type_value.items():
        
        # top bottom
        data_frame_features_index[frame_type_key][region_type_key] = {}
        
        # 遍历 region_type 层
        for row_num_key, row_num_value in region_type_value.items():
            
            # row 0 1
            data_frame_features_index[frame_type_key][region_type_key][row_num_key] = defaultdict(list)
            frame_count = 0
            # 遍历 row_num 层
            for col_num_key, group_data_item in row_num_value.items():
                # group_data_item 包含 FAR 和 frame_count
                frame_count += group_data_item["frame_count"]
                word_count = group_data_item["frame_count"] * 101 # 这里拿到word数
                end_index = start_index + word_count
                
                # 每次拿一帧的数据计算
                frame_index = 0
                for index in range(start_index, end_index, 101):
                    feature = get_feature(data_frame[index:index+101], "str")
                    
                    # 添加到字典
                    data_frame_features_index[frame_type_key][region_type_key][row_num_key][feature].append(
                        {
                            "FAR": {
                                "frame_type": group_data_item["FAR"]["frame_type"],
                                "region_type": group_data_item["FAR"]["region_type"],
                                "row_num": group_data_item["FAR"]["row_num"],
                                "col_num": group_data_item["FAR"]["col_num"]
                            },
                            "index": {
                                "start_index": index,
                                "end_index": index + 101
                            },
                            "feature": feature
                        }
                    )
                
                # 更新 start_index
                start_index = end_index
                
            # 完成一个row后，start_index需要向后跳两帧，这里有两帧的pad
            start_index += 101*2
            
            all_frame_count += frame_count

frame_count: 9448
data_frame: 955864


In [103]:
data_frame_features_index["frame_type_0"]["region_type_0"]["row_num_0"]

defaultdict(list,
            {'077a6de9ffaa89c2626a4dc887d001fd81cf8c41a9bf796acfba47ed385b4f79': [{'FAR': {'frame_type': 0,
                'region_type': 0,
                'row_num': 0,
                'col_num': 0},
               'index': {'start_index': 0, 'end_index': 101},
               'feature': '077a6de9ffaa89c2626a4dc887d001fd81cf8c41a9bf796acfba47ed385b4f79'},
              {'FAR': {'frame_type': 0,
                'region_type': 0,
                'row_num': 0,
                'col_num': 0},
               'index': {'start_index': 101, 'end_index': 202},
               'feature': '077a6de9ffaa89c2626a4dc887d001fd81cf8c41a9bf796acfba47ed385b4f79'},
              {'FAR': {'frame_type': 0,
                'region_type': 0,
                'row_num': 0,
                'col_num': 0},
               'index': {'start_index': 1010, 'end_index': 1111},
               'feature': '077a6de9ffaa89c2626a4dc887d001fd81cf8c41a9bf796acfba47ed385b4f79'},
              {'FAR': {'frame_ty

In [99]:
955864 - 955662

202

In [95]:
1616/(2*101)

8.0

In [83]:
2020 + 
1808
2020
1808
512
384
512
384

SyntaxError: invalid syntax (1332548728.py, line 1)